# Interactive Design Editor Implementation

This notebook implements a comprehensive design editor with features for element manipulation, transformations, grouping, and file operations.

## Import Required Libraries

In [ ]:
# Import standard libraries
import tkinter as tk
from tkinter import filedialog, colorchooser, messagebox
import os
import json
import copy
from PIL import Image, ImageTk
import numpy as np
import math

# For undo/redo functionality
from collections import deque

# Custom modules (placeholder imports)
# from element_manager import Element, TextElement, ImageElement, GroupElement
# from file_operations import FileManager
# from transformation_utilities import TransformationHelper

## Element Content Functionality

This section implements tools for adding and editing various types of content elements.

In [ ]:
class Element:
    """Base class for all design elements"""
    def __init__(self, x=0, y=0, width=100, height=100, rotation=0, opacity=1.0):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.rotation = rotation
        self.opacity = opacity
        self.selected = False
        self.id = id(self)  # Unique identifier
        
    def contains_point(self, x, y):
        """Check if element contains point (x,y)"""
        # Convert point to element's coordinate system
        cos_rot = math.cos(math.radians(-self.rotation))
        sin_rot = math.sin(math.radians(-self.rotation))
        
        # Translate to origin
        tx = x - self.x
        ty = y - self.y
        
        # Rotate
        rx = tx * cos_rot - ty * sin_rot
        ry = tx * sin_rot + ty * cos_rot
        
        # Check if point is inside
        return (-self.width/2 <= rx <= self.width/2) and (-self.height/2 <= ry <= self.height/2)
    
    def draw(self, canvas):
        """Draw element on canvas (to be implemented by subclasses)"""
        pass
    
    def get_properties(self):
        """Return dictionary of properties for panel display"""
        return {
            "x": self.x,
            "y": self.y,
            "width": self.width,
            "height": self.height,
            "rotation": self.rotation,
            "opacity": self.opacity
        }
        
    def set_property(self, prop_name, value):
        """Set property value"""
        if hasattr(self, prop_name):
            setattr(self, prop_name, value)
            return True
        return False


class TextElement(Element):
    """Element for text content"""
    def __init__(self, x=0, y=0, width=100, height=40, text="Text", font_family="Arial", 
                 font_size=12, font_color="#000000", rotation=0, opacity=1.0):
        super().__init__(x, y, width, height, rotation, opacity)
        self.text = text
        self.font_family = font_family
        self.font_size = font_size
        self.font_color = font_color
        
    def draw(self, canvas):
        # Implementation for drawing text on canvas
        pass
    
    def get_properties(self):
        props = super().get_properties()
        props.update({
            "text": self.text,
            "font_family": self.font_family,
            "font_size": self.font_size,
            "font_color": self.font_color
        })
        return props


class ImageElement(Element):
    """Element for image content"""
    def __init__(self, x=0, y=0, width=200, height=200, image_path="", rotation=0, opacity=1.0):
        super().__init__(x, y, width, height, rotation, opacity)
        self.image_path = image_path
        self.image = None
        self.load_image(image_path)
        
    def load_image(self, path):
        if os.path.exists(path):
            try:
                self.image = Image.open(path)
                # Adjust width/height to match aspect ratio
                if self.image.width > 0 and self.image.height > 0:
                    self.width = self.height * (self.image.width / self.image.height)
            except Exception as e:
                print(f"Error loading image: {e}")
                
    def draw(self, canvas):
        # Implementation for drawing image on canvas
        pass
    
    def get_properties(self):
        props = super().get_properties()
        props.update({
            "image_path": self.image_path,
        })
        return props


class CharacterElement(Element):
    """Element for character content from character library"""
    def __init__(self, x=0, y=0, width=100, height=100, character_id="", rotation=0, opacity=1.0):
        super().__init__(x, y, width, height, rotation, opacity)
        self.character_id = character_id
        
    def draw(self, canvas):
        # Implementation for drawing character on canvas
        pass
    
    def get_properties(self):
        props = super().get_properties()
        props.update({
            "character_id": self.character_id
        })
        return props

## Transformation Operations

This section implements functionality for transforming elements (moving, resizing, rotating, etc.).

In [ ]:
class TransformationManager:
    """Manages transformation operations for elements"""
    
    def __init__(self, canvas):
        self.canvas = canvas
        self._isDragging = False
        self._isResizing = False
        self._isRotating = False
        
        # Starting points for operations
        self._startX = 0
        self._startY = 0
        
        # Elements being transformed
        self.selected_elements = []
        
        # Reference to original element states for undoable operations
        self._original_states = {}
        
        # Which resize handle is being dragged (top-left, bottom-right, etc.)
        self._resize_handle = None
        
    def start_drag(self, x, y):
        """Start dragging selected elements"""
        if not self.selected_elements:
            return
            
        self._isDragging = True
        self._startX = x
        self._startY = y
        
        # Store original positions for undo
        self._original_states = {
            elem.id: {"x": elem.x, "y": elem.y} for elem in self.selected_elements
        }
        
    def drag(self, x, y):
        """Continue dragging selected elements"""
        if not self._isDragging:
            return
            
        dx = x - self._startX
        dy = y - self._startY
        
        for elem in self.selected_elements:
            elem.x += dx
            elem.y += dy
            
        self._startX = x
        self._startY = y
        
    def end_drag(self):
        """End dragging operation"""
        if not self._isDragging:
            return
            
        self._isDragging = False
        
        # Return the change data for undo/redo system
        new_states = {
            elem.id: {"x": elem.x, "y": elem.y} for elem in self.selected_elements
        }
        
        return {
            "type": "move",
            "elements": self.selected_elements,
            "before": self._original_states,
            "after": new_states
        }
        
    def start_resize(self, x, y, handle):
        """Start resizing selected elements"""
        if not self.selected_elements:
            return
            
        self._isResizing = True
        self._startX = x
        self._startY = y
        self._resize_handle = handle
        
        # Store original sizes for undo
        self._original_states = {
            elem.id: {"width": elem.width, "height": elem.height, "x": elem.x, "y": elem.y} 
            for elem in self.selected_elements
        }
        
    def resize(self, x, y):
        """Continue resizing selected elements"""
        if not self._isResizing:
            return
            
        dx = x - self._startX
        dy = y - self._startY
        
        for elem in self.selected_elements:
            # Logic depends on which handle is being dragged
            if self._resize_handle == "bottom-right":
                elem.width += dx
                elem.height += dy
            elif self._resize_handle == "top-left":
                old_width = elem.width
                old_height = elem.height
                
                elem.width -= dx
                elem.height -= dy
                
                # Adjust position to keep the bottom-right corner fixed
                elem.x += dx * (old_width / elem.width if elem.width != 0 else 0)
                elem.y += dy * (old_height / elem.height if elem.height != 0 else 0)
            # Add more handle cases as needed
            
        self._startX = x
        self._startY = y
        
    def end_resize(self):
        """End resizing operation"""
        if not self._isResizing:
            return
            
        self._isResizing = False
        
        # Return the change data for undo/redo system
        new_states = {
            elem.id: {"width": elem.width, "height": elem.height, "x": elem.x, "y": elem.y} 
            for elem in self.selected_elements
        }
        
        return {
            "type": "resize",
            "elements": self.selected_elements,
            "before": self._original_states,
            "after": new_states
        }
        
    def start_rotation(self, x, y):
        """Start rotating selected elements"""
        if not self.selected_elements:
            return
            
        self._isRotating = True
        self._startX = x
        self._startY = y
        
        # Store original rotations for undo
        self._original_states = {
            elem.id: {"rotation": elem.rotation} for elem in self.selected_elements
        }
        
    def rotate(self, x, y):
        """Continue rotating selected elements"""
        if not self._isRotating or not self.selected_elements:
            return
            
        # Calculate center of selection
        centers = [(elem.x, elem.y) for elem in self.selected_elements]
        center_x = sum(c[0] for c in centers) / len(centers)
        center_y = sum(c[1] for c in centers) / len(centers)
        
        # Calculate angles from center
        start_angle = math.atan2(self._startY - center_y, self._startX - center_x)
        current_angle = math.atan2(y - center_y, x - center_x)
        angle_diff = math.degrees(current_angle - start_angle)
        
        # Update rotations
        for elem in self.selected_elements:
            elem.rotation += angle_diff
            
        self._startX = x
        self._startY = y
        
    def end_rotation(self):
        """End rotation operation"""
        if not self._isRotating:
            return
            
        self._isRotating = False
        
        # Return the change data for undo/redo system
        new_states = {
            elem.id: {"rotation": elem.rotation} for elem in self.selected_elements
        }
        
        return {
            "type": "rotate",
            "elements": self.selected_elements,
            "before": self._original_states,
            "after": new_states
        }
        
    def set_opacity(self, opacity):
        """Set opacity for selected elements"""
        if not self.selected_elements:
            return
            
        # Store original opacities for undo
        original_states = {
            elem.id: {"opacity": elem.opacity} for elem in self.selected_elements
        }
        
        # Apply new opacity
        for elem in self.selected_elements:
            elem.opacity = opacity
            
        # Return the change data for undo/redo system
        new_states = {
            elem.id: {"opacity": elem.opacity} for elem in self.selected_elements
        }
        
        return {
            "type": "opacity",
            "elements": self.selected_elements,
            "before": original_states,
            "after": new_states
        }

## Grouping Operations

This section implements functionality for selecting, grouping, and ungrouping elements.

In [ ]:
class GroupElement(Element):
    """A group containing multiple elements"""
    
    def __init__(self, elements=None, x=0, y=0, width=0, height=0, rotation=0, opacity=1.0):
        # Calculate group bounds from elements
        if elements:
            min_x = min(e.x - e.width/2 for e in elements)
            min_y = min(e.y - e.height/2 for e in elements)
            max_x = max(e.x + e.width/2 for e in elements)
            max_y = max(e.y + e.height/2 for e in elements)
            
            x = (min_x + max_x) / 2
            y = (min_y + max_y) / 2
            width = max_x - min_x
            height = max_y - min_y
        
        super().__init__(x, y, width, height, rotation, opacity)
        self.elements = elements or []
        
    def draw(self, canvas):
        """Draw all elements in the group"""
        for element in self.elements:
            element.draw(canvas)
        
        # Draw group outline when selected
        if self.selected:
            # Implementation to draw selection border
            pass
            
    def contains_point(self, x, y):
        """Check if point is inside the group or any of its elements"""
        if super().contains_point(x, y):
            return True
            
        # Also check individual elements
        for element in self.elements:
            if element.contains_point(x, y):
                return True
                
        return False
        
    def get_properties(self):
        """Return group properties"""
        props = super().get_properties()
        props.update({
            "type": "group",
            "element_count": len(self.elements)
        })
        return props


class SelectionManager:
    """Manages element selection and grouping"""
    
    def __init__(self, canvas, elements):
        self.canvas = canvas
        self.elements = elements  # List of all elements on the canvas
        self.selected_elements = []  # List of currently selected elements
        
        # For marquee selection
        self.marquee_active = False
        self.marquee_start_x = 0
        self.marquee_start_y = 0
        self.marquee_current_x = 0
        self.marquee_current_y = 0
        
    def select_element(self, element, add_to_selection=False):
        """Select a single element"""
        if not add_to_selection:
            # Deselect all currently selected elements
            for elem in self.selected_elements:
                elem.selected = False
            self.selected_elements = []
            
        # Add to selection if not already selected
        if element not in self.selected_elements:
            element.selected = True
            self.selected_elements.append(element)
            
    def select_at_point(self, x, y, add_to_selection=False):
        """Select element at the given point"""
        # Check in reverse order (top to bottom) to select top elements first
        for element in reversed(self.elements):
            if element.contains_point(x, y):
                self.select_element(element, add_to_selection)
                return element
        
        if not add_to_selection:
            # Clear selection if clicked on empty space
            for elem in self.selected_elements:
                elem.selected = False
            self.selected_elements = []
            
        return None
        
    def start_marquee_selection(self, x, y):
        """Start marquee (rectangle) selection"""
        self.marquee_active = True
        self.marquee_start_x = x
        self.marquee_start_y = y
        self.marquee_current_x = x
        self.marquee_current_y = y
        
    def update_marquee_selection(self, x, y, add_to_selection=False):
        """Update marquee selection as mouse moves"""
        if not self.marquee_active:
            return
            
        self.marquee_current_x = x
        self.marquee_current_y = y
        
        # Calculate marquee rectangle
        min_x = min(self.marquee_start_x, self.marquee_current_x)
        min_y = min(self.marquee_start_y, self.marquee_current_y)
        max_x = max(self.marquee_start_x, self.marquee_current_x)
        max_y = max(self.marquee_start_y, self.marquee_current_y)
        
        # Check which elements are inside the marquee
        if not add_to_selection:
            # Clear current selection
            for elem in self.selected_elements:
                elem.selected = False
            self.selected_elements = []
            
        for element in self.elements:
            # Check if element's bounds intersect with marquee
            element_min_x = element.x - element.width/2
            element_min_y = element.y - element.height/2
            element_max_x = element.x + element.width/2
            element_max_y = element.y + element.height/2
            
            if (element_min_x < max_x and element_max_x > min_x and
                element_min_y < max_y and element_max_y > min_y):
                # Element is inside marquee - select it
                if element not in self.selected_elements:
                    element.selected = True
                    self.selected_elements.append(element)
        
    def end_marquee_selection(self):
        """End marquee selection"""
        self.marquee_active = False
        
    def group_selected_elements(self):
        """Group currently selected elements"""
        if len(self.selected_elements) < 2:
            return None  # Need at least 2 elements to form a group
            
        # Create a new group from selected elements
        elements_to_group = self.selected_elements.copy()
        
        # Remove elements from main list
        for elem in elements_to_group:
            elem.selected = False
            if elem in self.elements:
                self.elements.remove(elem)
                
        # Create the group
        group = GroupElement(elements_to_group)
        group.selected = True
        
        # Add group to elements and select only it
        self.elements.append(group)
        self.selected_elements = [group]
        
        return {
            "type": "group",
            "group": group,
            "elements": elements_to_group
        }
        
    def ungroup_selected(self):
        """Ungroup selected groups"""
        if not self.selected_elements:
            return None
            
        ungrouped_elements = []
        groups_to_remove = []
        
        for elem in self.selected_elements:
            if isinstance(elem, GroupElement):
                groups_to_remove.append(elem)
                
                # Extract elements from group
                for group_elem in elem.elements:
                    group_elem.selected = True
                    ungrouped_elements.append(group_elem)
                    self.elements.append(group_elem)
        
        # Remove groups from elements list
        for group in groups_to_remove:
            if group in self.elements:
                self.elements.remove(group)
                
        # Update selection
        self.selected_elements = [e for e in self.selected_elements 
                                 if e not in groups_to_remove]
        self.selected_elements.extend(ungrouped_elements)
        
        return {
            "type": "ungroup",
            "groups": groups_to_remove,
            "elements": ungrouped_elements
        }

## Auxiliary Features

This section implements grid, guidelines, and snapping functionality.

In [ ]:
class AuxiliaryTools:
    """Implements grid, guidelines, and snapping features"""
    
    def __init__(self, canvas):
        self.canvas = canvas
        
        # Grid settings
        self._gridSize = 20  # Size of grid cells
        self._showGrid = False
        self._snapToGrid = False
        
        # Guidelines
        self.guidelines = []  # List of guide positions (x or y coordinates)
        self._showGuides = True
        self._snapToGuides = True
        
        # Snapping settings
        self._snapDistance = 5  # Distance in pixels to trigger snapping
        
    def toggle_grid(self):
        """Toggle grid visibility"""
        self._showGrid = not self._showGrid
        
    def toggle_snap_to_grid(self):
        """Toggle snap to grid"""
        self._snapToGrid = not self._snapToGrid
        
    def set_grid_size(self, size):
        """Set grid size"""
        if size > 0:
            self._gridSize = size
            
    def toggle_guides(self):
        """Toggle guide visibility"""
        self._showGuides = not self._showGuides
        
    def toggle_snap_to_guides(self):
        """Toggle snap to guides"""
        self._snapToGuides = not self._snapToGuides
        
    def add_guide(self, position, orientation="horizontal"):
        """Add a guide at the specified position"""
        guide = {"position": position, "orientation": orientation}
        self.guidelines.append(guide)
        
    def remove_guide(self, position, orientation, tolerance=5):
        """Remove a guide near the specified position"""
        for i, guide in enumerate(self.guidelines):
            if (guide["orientation"] == orientation and 
                abs(guide["position"] - position) <= tolerance):
                del self.guidelines[i]
                return True
        return False
        
    def draw_grid(self):
        """Draw grid on canvas"""
        if not self._showGrid:
            return
            
        # Implementation for drawing grid
        # This would use self.canvas.create_line() for each grid line
        
    def draw_guides(self):
        """Draw guidelines on canvas"""
        if not self._showGuides:
            return
            
        # Implementation for drawing guidelines
        # This would use self.canvas.create_line() for each guideline
        
    def snap_position(self, x, y):
        """Snap position to grid/guides/elements and return the snapped position"""
        snapped_x = x
        snapped_y = y
        
        # Snap to grid
        if self._snapToGrid:
            snapped_x = round(x / self._gridSize) * self._gridSize
            snapped_y = round(y / self._gridSize) * self._gridSize
            
        # Snap to guides
        if self._snapToGuides:
            for guide in self.guidelines:
                if guide["orientation"] == "horizontal":
                    if abs(y - guide["position"]) <= self._snapDistance:
                        snapped_y = guide["position"]
                else:  # vertical guide
                    if abs(x - guide["position"]) <= self._snapDistance:
                        snapped_x = guide["position"]
                        
        return snapped_x, snapped_y
        
    def get_settings(self):
        """Return dictionary of current settings"""
        return {
            "gridSize": self._gridSize,
            "showGrid": self._showGrid,
            "snapToGrid": self._snapToGrid,
            "showGuides": self._showGuides,
            "snapToGuides": self._snapToGuides,
            "snapDistance": self._snapDistance
        }

## File Operations

This section implements save, load, export, and print functionalities.

# Design Editor Implementation
This notebook implements the features described in the design document for an editor application with various image editing and manipulation functionalities.

# Import Required Libraries
Import necessary libraries such as tkinter, PIL, and others for GUI and image processing.

In [ ]:
# Import Required Libraries
import tkinter as tk
from tkinter import ttk, filedialog, colorchooser
from PIL import Image, ImageTk, ImageDraw, ImageFont
import numpy as np
import json
import os
import sys
from collections import deque
import math

# Element Content Functionality
Implement tools for selecting characters from a font library, adding and editing images, and enabling drag-and-drop interactions.

In [ ]:
class ElementContent:
    def __init__(self, canvas):
        self.canvas = canvas
        self.font_library = {}  # Dictionary to store available fonts
        self.selected_element = None
        self.elements = []
        
    def load_fonts(self, font_directory):
        """Load fonts from the specified directory into the font library"""
        if not os.path.exists(font_directory):
            print(f"Font directory {font_directory} does not exist")
            return
        
        for file in os.listdir(font_directory):
            if file.endswith(('.ttf', '.otf')):
                font_path = os.path.join(font_directory, file)
                font_name = os.path.splitext(file)[0]
                self.font_library[font_name] = font_path
                print(f"Loaded font: {font_name}")
    
    def add_text(self, text, x, y, font_name=None, font_size=12, color="black"):
        """Add text element to the canvas"""
        font_path = self.font_library.get(font_name) if font_name else None
        
        text_element = {
            "type": "text",
            "content": text,
            "x": x,
            "y": y,
            "font_name": font_name,
            "font_size": font_size,
            "color": color,
            "rotation": 0,
            "opacity": 1.0,
            "id": len(self.elements)
        }
        
        self.elements.append(text_element)
        self._draw_text_element(text_element)
        return text_element["id"]
    
    def add_image(self, image_path, x, y):
        """Add image element to the canvas"""
        if not os.path.exists(image_path):
            print(f"Image path {image_path} does not exist")
            return None
        
        try:
            img = Image.open(image_path)
            width, height = img.size
            
            image_element = {
                "type": "image",
                "path": image_path,
                "x": x,
                "y": y,
                "width": width,
                "height": height,
                "rotation": 0,
                "opacity": 1.0,
                "id": len(self.elements)
            }
            
            self.elements.append(image_element)
            self._draw_image_element(image_element)
            return image_element["id"]
        except Exception as e:
            print(f"Error loading image: {e}")
            return None
    
    def _draw_text_element(self, element):
        """Draw text element on the canvas"""
        # Implementation would use tkinter canvas text
        canvas_id = self.canvas.create_text(
            element["x"], 
            element["y"],
            text=element["content"],
            fill=element["color"],
            font=(element["font_name"], element["font_size"]),
            tags=(f"element_{element['id']}", "text")
        )
        return canvas_id
    
    def _draw_image_element(self, element):
        """Draw image element on the canvas"""
        # Load image using PIL
        img = Image.open(element["path"])
        # Resize if needed
        img = img.resize((element["width"], element["height"]))
        # Apply rotation if needed
        if element["rotation"] != 0:
            img = img.rotate(element["rotation"], expand=True)
        # Apply opacity
        if element["opacity"] < 1.0:
            img = img.convert("RGBA")
            data = np.array(img)
            data[..., 3] = data[..., 3] * element["opacity"]
            img = Image.fromarray(data)
        
        # Convert to PhotoImage for tkinter
        photo = ImageTk.PhotoImage(img)
        # Store reference to prevent garbage collection
        element["_photo"] = photo
        
        # Create image on canvas
        canvas_id = self.canvas.create_image(
            element["x"], 
            element["y"],
            image=photo,
            tags=(f"element_{element['id']}", "image")
        )
        return canvas_id
    
    def setup_drag_drop(self):
        """Set up drag and drop functionality for elements"""
        self.canvas.tag_bind("text", "<ButtonPress-1>", self._on_element_select)
        self.canvas.tag_bind("image", "<ButtonPress-1>", self._on_element_select)
        self.canvas.bind("<B1-Motion>", self._on_element_drag)
        self.canvas.bind("<ButtonRelease-1>", self._on_element_release)
    
    def _on_element_select(self, event):
        """Handle element selection"""
        # Get the element ID from the canvas item
        item_id = self.canvas.find_withtag("current")[0]
        tags = self.canvas.gettags(item_id)
        
        for tag in tags:
            if tag.startswith("element_"):
                element_id = int(tag.split("_")[1])
                self.selected_element = element_id
                # Add highlight or selection indicator
                self._highlight_selected_element()
                break
    
    def _on_element_drag(self, event):
        """Handle element dragging"""
        if self.selected_element is not None:
            # Get the element
            element = self.elements[self.selected_element]
            # Update position
            element["x"] = event.x
            element["y"] = event.y
            # Redraw
            self.canvas.delete(f"element_{element['id']}")
            if element["type"] == "text":
                self._draw_text_element(element)
            elif element["type"] == "image":
                self._draw_image_element(element)
            # Update selection highlight
            self._highlight_selected_element()
    
    def _on_element_release(self, event):
        """Handle element release after drag"""
        # Element remains selected after drag
        pass
    
    def _highlight_selected_element(self):
        """Add highlight to the selected element"""
        if self.selected_element is not None:
            element = self.elements[self.selected_element]
            # Remove any existing selection highlights
            self.canvas.delete("selection_highlight")
            
            # Create a new highlight based on element type
            if element["type"] == "text":
                # Get the coordinates of the text
                bbox = self.canvas.bbox(f"element_{element['id']}")
                if bbox:
                    x1, y1, x2, y2 = bbox
                    # Draw selection rectangle
                    self.canvas.create_rectangle(
                        x1-5, y1-5, x2+5, y2+5,
                        outline="blue", width=2,
                        tags="selection_highlight"
                    )
            elif element["type"] == "image":
                # Get the coordinates of the image
                bbox = self.canvas.bbox(f"element_{element['id']}")
                if bbox:
                    x1, y1, x2, y2 = bbox
                    # Draw selection rectangle
                    self.canvas.create_rectangle(
                        x1, y1, x2, y2,
                        outline="blue", width=2,
                        tags="selection_highlight"
                    )

# Transformation Operations
Define variables for transformation states (_isDragging, _isResizing, _isRotating) and implement position, size, rotation, and opacity adjustments.

In [ ]:
class TransformationOperations:
    def __init__(self, canvas, element_content):
        self.canvas = canvas
        self.element_content = element_content
        
        # Transformation states
        self._isDragging = False
        self._isResizing = False
        self._isRotating = False
        
        # Handles for resize and rotate
        self.resize_handles = []
        self.rotate_handle = None
        
        # Starting positions for transformations
        self.start_x = 0
        self.start_y = 0
        self.start_width = 0
        self.start_height = 0
        self.start_angle = 0
        
    def setup_transformation_controls(self):
        """Set up event bindings for transformation operations"""
        # Bind drag event (already handled in ElementContent)
        
        # Bind resize and rotate handle events
        self.canvas.tag_bind("resize_handle", "<ButtonPress-1>", self._on_resize_start)
        self.canvas.tag_bind("rotate_handle", "<ButtonPress-1>", self._on_rotate_start)
        
        self.canvas.bind("<B1-Motion>", self._on_transform_motion)
        self.canvas.bind("<ButtonRelease-1>", self._on_transform_release)
        
    def create_transform_handles(self):
        """Create transform handles around the selected element"""
        if self.element_content.selected_element is None:
            return
            
        # Clear previous handles
        self._clear_transform_handles()
        
        element = self.element_content.elements[self.element_content.selected_element]
        bbox = self.canvas.bbox(f"element_{element['id']}")
        
        if not bbox:
            return
            
        x1, y1, x2, y2 = bbox
        
        # Create resize handles at corners and midpoints
        handle_positions = [
            (x1, y1),  # Top-left
            ((x1+x2)/2, y1),  # Top-middle
            (x2, y1),  # Top-right
            (x2, (y1+y2)/2),  # Middle-right
            (x2, y2),  # Bottom-right
            ((x1+x2)/2, y2),  # Bottom-middle
            (x1, y2),  # Bottom-left
            (x1, (y1+y2)/2)   # Middle-left
        ]
        
        for x, y in handle_positions:
            handle = self.canvas.create_rectangle(
                x-5, y-5, x+5, y+5,
                fill="white", outline="blue",
                tags=("transform_control", "resize_handle")
            )
            self.resize_handles.append(handle)
        
        # Create rotation handle above the top-middle point
        rotation_x = (x1 + x2) / 2
        rotation_y = y1 - 25
        rotate_handle = self.canvas.create_oval(
            rotation_x-5, rotation_y-5, rotation_x+5, rotation_y+5,
            fill="green", outline="blue",
            tags=("transform_control", "rotate_handle")
        )
        self.rotate_handle = rotate_handle
        
        # Create a line connecting the rotation handle to the object
        self.canvas.create_line(
            rotation_x, rotation_y, rotation_x, y1,
            fill="blue", width=1,
            tags=("transform_control", "rotate_line")
        )
        
    def _clear_transform_handles(self):
        """Clear transformation handles"""
        self.canvas.delete("transform_control")
        self.resize_handles = []
        self.rotate_handle = None
        
    def _on_resize_start(self, event):
        """Handle start of resize operation"""
        self._isResizing = True
        self.start_x = event.x
        self.start_y = event.y
        
        if self.element_content.selected_element is not None:
            element = self.element_content.elements[self.element_content.selected_element]
            if element["type"] == "image":
                self.start_width = element["width"]
                self.start_height = element["height"]
                
    def _on_rotate_start(self, event):
        """Handle start of rotation operation"""
        self._isRotating = True
        self.start_x = event.x
        self.start_y = event.y
        
        if self.element_content.selected_element is not None:
            element = self.element_content.elements[self.element_content.selected_element]
            self.start_angle = element.get("rotation", 0)
            
            # Calculate the center of the element (pivot point)
            bbox = self.canvas.bbox(f"element_{element['id']}")
            if bbox:
                x1, y1, x2, y2 = bbox
                self.pivot_x = (x1 + x2) / 2
                self.pivot_y = (y1 + y2) / 2
        
    def _on_transform_motion(self, event):
        """Handle mouse motion during transformation operations"""
        if self._isResizing:
            self._resize_element(event)
        elif self._isRotating:
            self._rotate_element(event)
        # Dragging is handled in ElementContent class
        
    def _on_transform_release(self, event):
        """Handle release of transformation operations"""
        self._isResizing = False
        self._isRotating = False
        
        # Recreate transformation handles if element is still selected
        if self.element_content.selected_element is not None:
            self.create_transform_handles()
            
    def _resize_element(self, event):
        """Resize the selected element"""
        if self.element_content.selected_element is None:
            return
            
        element = self.element_content.elements[self.element_content.selected_element]
        
        # Calculate the change in position
        dx = event.x - self.start_x
        dy = event.y - self.start_y
        
        if element["type"] == "image":
            # Update dimensions based on the handle being dragged
            # This is a simplified implementation
            new_width = max(10, self.start_width + dx)
            new_height = max(10, self.start_height + dy)
            
            element["width"] = new_width
            element["height"] = new_height
            
            # Redraw the element
            self.canvas.delete(f"element_{element['id']}")
            self.element_content._draw_image_element(element)
            
    def _rotate_element(self, event):
        """Rotate the selected element"""
        if self.element_content.selected_element is None:
            return
            
        element = self.element_content.elements[self.element_content.selected_element]
        
        # Calculate angle between center of element and current mouse position
        angle = math.degrees(math.atan2(event.y - self.pivot_y, event.x - self.pivot_x))
        # Adjust to standard 0-360 degrees
        angle = (angle + 90) % 360
        
        element["rotation"] = angle
        
        # Redraw the element
        self.canvas.delete(f"element_{element['id']}")
        if element["type"] == "text":
            self.element_content._draw_text_element(element)
        elif element["type"] == "image":
            self.element_content._draw_image_element(element)
            
    def set_opacity(self, opacity):
        """Set opacity of the selected element"""
        if self.element_content.selected_element is None:
            return
            
        element = self.element_content.elements[self.element_content.selected_element]
        element["opacity"] = max(0.0, min(1.0, opacity))
        
        # Redraw the element
        self.canvas.delete(f"element_{element['id']}")
        if element["type"] == "text":
            self.element_content._draw_text_element(element)
        elif element["type"] == "image":
            self.element_content._draw_image_element(element)

# Grouping Operations
Implement multi-select, group, and ungroup functionalities for elements.

In [ ]:
class GroupingOperations:
    def __init__(self, canvas, element_content):
        self.canvas = canvas
        self.element_content = element_content
        
        # For multiple selection
        self.selected_elements = set()
        self.groups = []  # List of element groups
        
        # Selection rectangle
        self.selection_start_x = 0
        self.selection_start_y = 0
        self.selection_rectangle = None
        
    def setup_multi_select(self):
        """Set up multi-select functionality"""
        # Add Shift+Click for individual element selection
        self.canvas.tag_bind("text", "<Shift-ButtonPress-1>", self._on_shift_select)
        self.canvas.tag_bind("image", "<Shift-ButtonPress-1>", self._on_shift_select)
        
        # Add drag selection rectangle
        self.canvas.bind("<Control-ButtonPress-1>", self._on_selection_start)
        self.canvas.bind("<Control-B1-Motion>", self._on_selection_drag)
        self.canvas.bind("<Control-ButtonRelease-1>", self._on_selection_release)
        
    def _on_shift_select(self, event):
        """Handle shift+click selection for multiple elements"""
        # Get the element ID from the canvas item
        item_id = self.canvas.find_withtag("current")[0]
        tags = self.canvas.gettags(item_id)
        
        for tag in tags:
            if tag.startswith("element_"):
                element_id = int(tag.split("_")[1])
                
                # Toggle selection
                if element_id in self.selected_elements:
                    self.selected_elements.remove(element_id)
                else:
                    self.selected_elements.add(element_id)
                    
                # Update highlight
                self._highlight_selected_elements()
                break
    
    def _on_selection_start(self, event):
        """Start drawing a selection rectangle"""
        self.selection_start_x = event.x
        self.selection_start_y = event.y
        
        # Create a new selection rectangle
        self.selection_rectangle = self.canvas.create_rectangle(
            self.selection_start_x, self.selection_start_y,
            self.selection_start_x, self.selection_start_y,
            outline="blue", dash=(4, 4),
            tags="selection_rectangle"
        )
    
    def _on_selection_drag(self, event):
        """Update the selection rectangle during dragging"""
        if self.selection_rectangle:
            self.canvas.coords(
                self.selection_rectangle,
                self.selection_start_x, self.selection_start_y,
                event.x, event.y
            )
    
    def _on_selection_release(self, event):
        """Finalize the selection rectangle and select elements within it"""
        if not self.selection_rectangle:
            return
            
        # Get the coordinates of the selection rectangle
        x1 = min(self.selection_start_x, event.x)
        y1 = min(self.selection_start_y, event.y)
        x2 = max(self.selection_start_x, event.x)
        y2 = max(self.selection_start_y, event.y)
        
        # Reset selected elements if not holding Shift
        if not event.state & 0x1:  # Check if Shift is not being held
            self.selected_elements.clear()
        
        # Find all elements within the selection rectangle
        for i, element in enumerate(self.element_content.elements):
            # Get the bounding box of the element
            element_id = f"element_{element['id']}"
            bbox = self.canvas.bbox(element_id)
            
            if not bbox:
                continue
                
            e_x1, e_y1, e_x2, e_y2 = bbox
            
            # Check if the element is within the selection rectangle
            if (x1 <= e_x1 and e_x2 <= x2 and
                y1 <= e_y1 and e_y2 <= y2):
                self.selected_elements.add(element["id"])
        
        # Highlight selected elements
        self._highlight_selected_elements()
        
        # Remove the selection rectangle
        self.canvas.delete(self.selection_rectangle)
        self.selection_rectangle = None
    
    def _highlight_selected_elements(self):
        """Highlight all selected elements"""
        # Remove existing highlights
        self.canvas.delete("selection_highlight")
        
        # Highlight each selected element
        for element_id in self.selected_elements:
            element = self.element_content.elements[element_id]
            bbox = self.canvas.bbox(f"element_{element['id']}")
            
            if bbox:
                x1, y1, x2, y2 = bbox
                # Draw selection rectangle
                self.canvas.create_rectangle(
                    x1-5, y1-5, x2+5, y2+5,
                    outline="blue", width=2,
                    dash=(2, 4),
                    tags="selection_highlight"
                )
    
    def group_selected_elements(self):
        """Group the currently selected elements"""
        if len(self.selected_elements) < 2:
            print("At least two elements must be selected to create a group")
            return
        
        # Create a new group
        group = {
            "id": len(self.groups),
            "element_ids": list(self.selected_elements),
            "type": "group"
        }
        
        self.groups.append(group)
        
        # Add the group tag to all elements in the group
        for element_id in self.selected_elements:
            # Update the canvas item tags
            canvas_item = self.canvas.find_withtag(f"element_{element_id}")
            if canvas_item:
                current_tags = self.canvas.gettags(canvas_item)
                new_tags = [tag for tag in current_tags if not tag.startswith("group_")]
                new_tags.append(f"group_{group['id']}")
                self.canvas.itemconfig(canvas_item, tags=tuple(new_tags))
        
        print(f"Created group {group['id']} with {len(self.selected_elements)} elements")
        
        # Clear selection after grouping
        self.selected_elements.clear()
        self.canvas.delete("selection_highlight")
    
    def ungroup_selected_group(self):
        """Ungroup the currently selected group"""
        # First check if a group is selected
        selected_groups = set()
        
        for element_id in self.selected_elements:
            element = self.element_content.elements[element_id]
            canvas_item = self.canvas.find_withtag(f"element_{element['id']}")
            if canvas_item:
                tags = self.canvas.gettags(canvas_item)
                for tag in tags:
                    if tag.startswith("group_"):
                        group_id = int(tag.split("_")[1])
                        selected_groups.add(group_id)
        
        if not selected_groups:
            print("No groups selected")
            return
        
        # Ungroup each selected group
        for group_id in selected_groups:
            # Find the group
            for i, group in enumerate(self.groups):
                if group["id"] == group_id:
                    # Remove the group tag from all elements in the group
                    for element_id in group["element_ids"]:
                        canvas_item = self.canvas.find_withtag(f"element_{element_id}")
                        if canvas_item:
                            current_tags = self.canvas.gettags(canvas_item)
                            new_tags = [tag for tag in current_tags if not tag == f"group_{group_id}"]
                            self.canvas.itemconfig(canvas_item, tags=tuple(new_tags))
                    
                    # Remove the group
                    self.groups.pop(i)
                    print(f"Ungrouped group {group_id}")
                    break
        
        # Clear selection after ungrouping
        self.selected_elements.clear()
        self.canvas.delete("selection_highlight")

# Auxiliary Features
Add guidelines, grid display functionality using _gridSize, and implement smart snapping for alignment.

In [ ]:
class AuxiliaryFeatures:
    def __init__(self, canvas):
        self.canvas = canvas
        self._gridSize = 20  # Default grid size
        self._showGrid = False
        self._showGuidelines = False
        self._enableSnapping = False
        
        # Grid and guidelines
        self.grid_lines = []
        self.guidelines = []
        
        # Canvas dimensions
        self.canvas_width = 800
        self.canvas_height = 600
    
    def toggle_grid(self):
        """Toggle grid display"""
        self._showGrid = not self._showGrid
        
        if self._showGrid:
            self._draw_grid()
        else:
            self._clear_grid()
    
    def set_grid_size(self, size):
        """Set the grid size"""
        self._gridSize = max(5, min(100, size))  # Limit grid size between 5 and 100
        
        if self._showGrid:
            self._clear_grid()
            self._draw_grid()
    
    def _draw_grid(self):
        """Draw grid lines on the canvas"""
        self._clear_grid()
        
        # Get canvas dimensions
        width = self.canvas_width
        height = self.canvas_height
        
        # Draw vertical grid lines
        for x in range(0, width, self._gridSize):
            line = self.canvas.create_line(
                x, 0, x, height,
                fill="#DDDDDD", width=1,
                tags="grid_line"
            )
            self.grid_lines.append(line)
        
        # Draw horizontal grid lines
        for y in range(0, height, self._gridSize):
            line = self.canvas.create_line(
                0, y, width, y,
                fill="#DDDDDD", width=1,
                tags="grid_line"
            )
            self.grid_lines.append(line)
    
    def _clear_grid(self):
        """Clear grid lines from the canvas"""
        self.canvas.delete("grid_line")
        self.grid_lines = []
    
    def toggle_guidelines(self):
        """Toggle guidelines display"""
        self._showGuidelines = not self._showGuidelines
        
        if not self._showGuidelines:
            self._clear_guidelines()
    
    def _clear_guidelines(self):
        """Clear guidelines from the canvas"""
        self.canvas.delete("guideline")
        self.guidelines = []
    
    def create_guideline(self, x=None, y=None):
        """Create a new guideline at the specified coordinates"""
        if not self._showGuidelines:
            return
        
        width = self.canvas_width
        height = self.canvas_height
        
        if x is not None:
            # Create vertical guideline
            line = self.canvas.create_line(
                x, 0, x, height,
                fill="#FF0000", width=1, dash=(4, 4),
                tags="guideline"
            )
            self.guidelines.append({
                "line": line,
                "type": "vertical",
                "position": x
            })
        
        if y is not None:
            # Create horizontal guideline
            line = self.canvas.create_line(
                0, y, width, y,
                fill="#FF0000", width=1, dash=(4, 4),
                tags="guideline"
            )
            self.guidelines.append({
                "line": line,
                "type": "horizontal",
                "position": y
            })
    
    def toggle_snapping(self):
        """Toggle snapping functionality"""
        self._enableSnapping = not self._enableSnapping
    
    def snap_to_grid(self, x, y):
        """Snap a coordinate to the nearest grid point"""
        if not self._enableSnapping:
            return x, y
        
        # Calculate the nearest grid point
        snapped_x = round(x / self._gridSize) * self._gridSize
        snapped_y = round(y / self._gridSize) * self._gridSize
        
        return snapped_x, snapped_y
    
    def snap_to_guidelines(self, x, y, threshold=10):
        """Snap a coordinate to the nearest guideline if within threshold"""
        if not self._enableSnapping or not self._showGuidelines:
            return x, y
        
        snapped_x = x
        snapped_y = y
        
        # Check vertical guidelines for x coordinate
        for guideline in self.guidelines:
            if guideline["type"] == "vertical":
                if abs(x - guideline["position"]) <= threshold:
                    snapped_x = guideline["position"]
                    # Highlight the guideline briefly
                    self.canvas.itemconfig(guideline["line"], width=2, fill="#FF8800")
                    self.canvas.after(500, lambda gl=guideline["line"]: self.canvas.itemconfig(gl, width=1, fill="#FF0000"))
            
            if guideline["type"] == "horizontal":
                if abs(y - guideline["position"]) <= threshold:
                    snapped_y = guideline["position"]
                    # Highlight the guideline briefly
                    self.canvas.itemconfig(guideline["line"], width=2, fill="#FF8800")
                    self.canvas.after(500, lambda gl=guideline["line"]: self.canvas.itemconfig(gl, width=1, fill="#FF0000"))
        
        return snapped_x, snapped_y
    
    def smart_align_elements(self, element_ids):
        """Smart align multiple elements based on their positions"""
        if len(element_ids) < 2:
            return
        
        # Get the coordinates of all elements
        element_coords = []
        for element_id in element_ids:
            bbox = self.canvas.bbox(f"element_{element_id}")
            if bbox:
                x1, y1, x2, y2 = bbox
                center_x = (x1 + x2) / 2
                center_y = (y1 + y2) / 2
                element_coords.append({
                    "id": element_id,
                    "x1": x1, "y1": y1,
                    "x2": x2, "y2": y2,
                    "center_x": center_x,
                    "center_y": center_y
                })
        
        if not element_coords:
            return
        
        # Calculate average center X and Y
        avg_center_x = sum(e["center_x"] for e in element_coords) / len(element_coords)
        avg_center_y = sum(e["center_y"] for e in element_coords) / len(element_coords)
        
        # Create guidelines at these positions
        self.create_guideline(x=avg_center_x)
        self.create_guideline(y=avg_center_y)

# File Operations
Implement save, save as, export, and print functionalities for the project.

In [ ]:
class FileOperations:
    def __init__(self, canvas, element_content):
        self.canvas = canvas
        self.element_content = element_content
        self.current_file_path = None
    
    def new_project(self):
        """Create a new project"""
        # Clear the canvas
        self.canvas.delete("all")
        
        # Clear element list
        self.element_content.elements = []
        self.element_content.selected_element = None
        
        # Reset file path
        self.current_file_path = None
        
        print("Created new project")
    
    def save_project(self):
        """Save the current project"""
        if self.current_file_path:
            self._save_to_file(self.current_file_path)
        else:
            self.save_project_as()
    
    def save_project_as(self):
        """Save the current project with a new file name"""
        file_path = filedialog.asksaveasfilename(
            defaultextension=".json",
            filetypes=[("Design Editor Files", "*.json"), ("All Files", "*.*")]
        )
        
        if not file_path:
            return
        
        self._save_to_file(file_path)
        self.current_file_path = file_path
    
    def _save_to_file(self, file_path):
        """Save project data to a file"""
        try:
            # Create a data structure to represent the project
            project_data = {
                "version": "1.0",
                "elements": self.element_content.elements,
                "canvas_width": self.canvas.winfo_width(),
                "canvas_height": self.canvas.winfo_height()
            }
            
            # Save to file
            with open(file_path, 'w') as f:
                json.dump(project_data, f, indent=2)
            
            print(f"Project saved to {file_path}")
            return True
        except Exception as e:
            print(f"Error saving project: {e}")
            return False
    
    def open_project(self):
        """Open a project from file"""
        file_path = filedialog.askopenfilename(
            filetypes=[("Design Editor Files", "*.json"), ("All Files", "*.*")]
        )
        
        if not file_path:
            return
        
        try:
            with open(file_path, 'r') as f:
                project_data = json.load(f)
            
            # Validate file format
            if "version" not in project_data or "elements" not in project_data:
                raise ValueError("Invalid project file format")
            
            # Clear current project
            self.new_project()
            
            # Load elements
            self.element_content.elements = project_data["elements"]
            
            # Draw all elements
            for element in self.element_content.elements:
                if element["type"] == "text":
                    self.element_content._draw_text_element(element)
                elif element["type"] == "image":
                    self.element_content._draw_image_element(element)
            
            # Update file path
            self.current_file_path = file_path
            
            print(f"Project loaded from {file_path}")
            return True
        except Exception as e:
            print(f"Error opening project: {e}")
            return False
    
    def export_as_image(self):
        """Export the canvas as an image"""
        file_path = filedialog.asksaveasfilename(
            defaultextension=".png",
            filetypes=[
                ("PNG Image", "*.png"),
                ("JPEG Image", "*.jpg;*.jpeg"),
                ("All Files", "*.*")
            ]
        )
        
        if not file_path:
            return
        
        try:
            # Get canvas dimensions
            canvas_width = self.canvas.winfo_width()
            canvas_height = self.canvas.winfo_height()
            
            # Create a new image
            image = Image.new("RGB", (canvas_width, canvas_height), "white")
            draw = ImageDraw.Draw(image)
            
            # Draw each element
            for element in self.element_content.elements:
                if element["type"] == "text":
                    # Draw text
                    font = ImageFont.truetype(
                        element.get("font_name", "Arial"),
                        element.get("font_size", 12)
                    )
                    draw.text(
                        (element["x"], element["y"]),
                        element["content"],
                        fill=element["color"],
                        font=font
                    )
                elif element["type"] == "image":
                    # Draw image
                    try:
                        img = Image.open(element["path"])
                        img = img.resize((element["width"], element["height"]))
                        
                        if element["rotation"] != 0:
                            img = img.rotate(element["rotation"], expand=True)
                        
                        # Paste the image
                        image.paste(
                            img,
                            (int(element["x"] - img.width/2), int(element["y"] - img.height/2))
                        )
                    except Exception as e:
                        print(f"Error drawing image: {e}")
            
            # Save the image
            image.save(file_path)
            print(f"Image exported to {file_path}")
            return True
        except Exception as e:
            print(f"Error exporting image: {e}")
            return False
    
    def print_project(self):
        """Print the current project"""
        # First export as a temporary image
        import tempfile
        import os
        
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
        temp_file.close()
        
        try:
            # Export to temporary file
            if self.export_as_image_to_path(temp_file.name):
                # Open the file with the default image viewer
                os.startfile(temp_file.name, "print")
                print("Print job sent to printer")
            else:
                print("Failed to prepare image for printing")
        except Exception as e:
            print(f"Error printing project: {e}")
        finally:
            # Schedule file for deletion
            try:
                os.unlink(temp_file.name)
            except:
                pass
    
    def export_as_image_to_path(self, file_path):
        """Export the canvas as an image to the specified path"""
        try:
            # Get canvas dimensions
            canvas_width = self.canvas.winfo_width()
            canvas_height = self.canvas.winfo_height()
            
            # Create a new image
            image = Image.new("RGB", (canvas_width, canvas_height), "white")
            draw = ImageDraw.Draw(image)
            
            # Draw each element
            for element in self.element_content.elements:
                if element["type"] == "text":
                    # Draw text
                    font = ImageFont.truetype(
                        element.get("font_name", "Arial"),
                        element.get("font_size", 12)
                    )
                    draw.text(
                        (element["x"], element["y"]),
                        element["content"],
                        fill=element["color"],
                        font=font
                    )
                elif element["type"] == "image":
                    # Draw image
                    try:
                        img = Image.open(element["path"])
                        img = img.resize((element["width"], element["height"]))
                        
                        if element["rotation"] != 0:
                            img = img.rotate(element["rotation"], expand=True)
                        
                        # Paste the image
                        image.paste(
                            img,
                            (int(element["x"] - img.width/2), int(element["y"] - img.height/2))
                        )
                    except Exception as e:
                        print(f"Error drawing image: {e}")
            
            # Save the image
            image.save(file_path)
            return True
        except Exception as e:
            print(f"Error exporting image: {e}")
            return False

# Undo/Redo
Implement operation history tracking and undo/redo functionality.

In [ ]:
class HistoryManager:
    def __init__(self, canvas, element_content):
        self.canvas = canvas
        self.element_content = element_content
        
        # Undo and redo stacks
        self.undo_stack = deque(maxlen=20)  # Limit to 20 operations
        self.redo_stack = deque(maxlen=20)
        
        # Track whether we're currently performing an undo/redo operation
        self.is_undoing = False
        self.is_redoing = False
    
    def record_state(self, operation_name):
        """Record the current state for undo/redo"""
        if self.is_undoing or self.is_redoing:
            return
        
        # Clone the current elements list
        current_state = {
            "elements": [element.copy() for element in self.element_content.elements],
            "operation": operation_name
        }
        
        # Add to undo stack
        self.undo_stack.append(current_state)
        
        # Clear redo stack when a new action is performed
        self.redo_stack.clear()
    
    def undo(self):
        """Undo the last operation"""
        if not self.undo_stack:
            print("Nothing to undo")
            return
        
        # Mark that we're undoing
        self.is_undoing = True
        
        # Get the current state for redo
        current_state = {
            "elements": [element.copy() for element in self.element_content.elements],
            "operation": "state_before_undo"
        }
        
        # Add current state to redo stack
        self.redo_stack.append(current_state)
        
        # Get the previous state from undo stack
        previous_state = self.undo_stack.pop()
        
        # Restore the previous state
        self._restore_state(previous_state["elements"])
        
        print(f"Undid operation: {previous_state['operation']}")
        
        # Reset undoing flag
        self.is_undoing = False
    
    def redo(self):
        """Redo the last undone operation"""
        if not self.redo_stack:
            print("Nothing to redo")
            return
        
        # Mark that we're redoing
        self.is_redoing = True
        
        # Get the current state for undo
        current_state = {
            "elements": [element.copy() for element in self.element_content.elements],
            "operation": "state_before_redo"
        }
        
        # Add current state to undo stack
        self.undo_stack.append(current_state)
        
        # Get the next state from redo stack
        next_state = self.redo_stack.pop()
        
        # Restore the next state
        self._restore_state(next_state["elements"])
        
        print(f"Redid operation")
        
        # Reset redoing flag
        self.is_redoing = False
    
    def _restore_state(self, elements_state):
        """Restore the canvas to a given state"""
        # Clear the canvas
        self.canvas.delete("all")
        
        # Update elements list
        self.element_content.elements = elements_state
        
        # Redraw all elements
        for element in self.element_content.elements:
            if element["type"] == "text":
                self.element_content._draw_text_element(element)
            elif element["type"] == "image":
                self.element_content._draw_image_element(element)

# Page Management
Use PageThumbnailStrip and PageOperations components to add, delete, and reorder pages.

In [ ]:
class PageManager:
    def __init__(self, main_canvas, element_content, history_manager):
        self.main_canvas = main_canvas
        self.element_content = element_content
        self.history_manager = history_manager
        
        # Pages data structure
        self.pages = []
        self.current_page_index = -1
        
        # Thumbnail canvas
        self.thumbnail_canvas = None
        self.thumbnail_width = 100
        self.thumbnail_height = 150
        self.thumbnail_margin = 10
        
    def create_thumbnail_strip(self, parent):
        """Create a canvas for page thumbnails"""
        self.thumbnail_canvas = tk.Canvas(
            parent,
            width=parent.winfo_width(),
            height=self.thumbnail_height + 2 * self.thumbnail_margin,
            bg="#F0F0F0"
        )
        
        # Bind click event to select page
        self.thumbnail_canvas.bind("<ButtonPress-1>", self._on_thumbnail_click)
        
        return self.thumbnail_canvas
    
    def add_page(self):
        """Add a new page to the project"""
        # Create a new page
        page = {
            "id": len(self.pages),
            "name": f"Page {len(self.pages) + 1}",
            "elements": [],
            "thumbnail": None
        }
        
        # Add to pages list
        self.pages.append(page)
        
        # Update thumbnails
        self._update_thumbnails()
        
        # Switch to new page if this is the first page
        if len(self.pages) == 1:
            self.switch_to_page(0)
        else:
            # Record history for page addition
            self.history_manager.record_state("add_page")
        
        print(f"Added page {page['name']}")
    
    def delete_page(self, page_index=None):
        """Delete a page from the project"""
        if not self.pages:
            print("No pages to delete")
            return
        
        # If no index specified, delete current page
        if page_index is None:
            page_index = self.current_page_index
        
        # Validate index
        if page_index < 0 or page_index >= len(self.pages):
            print(f"Invalid page index: {page_index}")
            return
        
        # Record history before deletion
        self.history_manager.record_state("delete_page")
        
        # Remove the page
        deleted_page = self.pages.pop(page_index)
        print(f"Deleted page {deleted_page['name']}")
        
        # Update thumbnail display
        self._update_thumbnails()
        
        # Switch to another page if necessary
        if not self.pages:
            # No pages left, clear canvas
            self.main_canvas.delete("all")
            self.element_content.elements = []
            self.current_page_index = -1
        elif self.current_page_index == page_index:
            # Deleted the current page
            new_index = min(page_index, len(self.pages) - 1)
            self.switch_to_page(new_index)
        elif self.current_page_index > page_index:
            # Adjust current page index
            self.current_page_index -= 1
    
    def switch_to_page(self, page_index):
        """Switch to the specified page"""
        if page_index < 0 or page_index >= len(self.pages):
            print(f"Invalid page index: {page_index}")
            return
        
        # If switching to the same page, do nothing
        if page_index == self.current_page_index:
            return
        
        # Save current page state
        if self.current_page_index >= 0:
            self.pages[self.current_page_index]["elements"] = [
                element.copy() for element in self.element_content.elements
            ]
            
            # Create thumbnail for the current page
            self._create_thumbnail(self.current_page_index)
        
        # Load the new page
        self.current_page_index = page_index
        new_page = self.pages[page_index]
        
        # Clear the canvas
        self.main_canvas.delete("all")
        
        # Load elements
        self.element_content.elements = [
            element.copy() for element in new_page["elements"]
        ]
        
        # Draw all elements
        for element in self.element_content.elements:
            if element["type"] == "text":
                self.element_content._draw_text_element(element)
            elif element["type"] == "image":
                self.element_content._draw_image_element(element)
        
        # Update thumbnails to highlight current page
        self._update_thumbnails()
        
        print(f"Switched to {new_page['name']}")
    
    def reorder_pages(self, from_index, to_index):
        """Reorder pages by moving a page from one index to another"""
        if from_index < 0 or from_index >= len(self.pages):
            print(f"Invalid source index: {from_index}")
            return
        
        if to_index < 0 or to_index >= len(self.pages):
            print(f"Invalid destination index: {to_index}")
            return
        
        # Record history
        self.history_manager.record_state("reorder_pages")
        
        # Move the page
        page = self.pages.pop(from_index)
        self.pages.insert(to_index, page)
        
        # Update current page index if necessary
        if self.current_page_index == from_index:
            self.current_page_index = to_index
        elif from_index < self.current_page_index <= to_index:
            self.current_page_index -= 1
        elif to_index <= self.current_page_index < from_index:
            self.current_page_index += 1
        
        # Update thumbnails
        self._update_thumbnails()
        
        print(f"Moved page from position {from_index + 1} to {to_index + 1}")
    
    def _create_thumbnail(self, page_index):
        """Create a thumbnail image for a page"""
        page = self.pages[page_index]
        
        # Create an image for the thumbnail
        thumbnail = Image.new("RGB", (self.thumbnail_width, self.thumbnail_height), "white")
        draw = ImageDraw.Draw(thumbnail)
        
        # Scale factor for drawing elements
        scale_x = self.thumbnail_width / self.main_canvas.winfo_width()
        scale_y = self.thumbnail_height / self.main_canvas.winfo_height()
        
        # Draw elements
        for element in page["elements"]:
            if element["type"] == "text":
                # Draw text (simplified)
                draw.text(
                    (element["x"] * scale_x, element["y"] * scale_y),
                    element["content"],
                    fill=element["color"],
                )
            elif element["type"] == "image":
                # Draw a rectangle representing the image
                x = element["x"] * scale_x
                y = element["y"] * scale_y
                width = element["width"] * scale_x
                height = element["height"] * scale_y
                
                draw.rectangle(
                    (x - width/2, y - height/2, x + width/2, y + height/2),
                    outline="black"
                )
        
        # Save thumbnail
        page["thumbnail"] = thumbnail
    
    def _update_thumbnails(self):
        """Update the thumbnail display"""
        if not self.thumbnail_canvas:
            return
        
        # Clear the canvas
        self.thumbnail_canvas.delete("all")
        
        # Calculate available width
        available_width = self.thumbnail_canvas.winfo_width()
        
        # Draw thumbnails
        x = self.thumbnail_margin
        for i, page in enumerate(self.pages):
            # Create thumbnail if it doesn't exist
            if not page.get("thumbnail"):
                self._create_thumbnail(i)
            
            # Highlight if current page
            if i == self.current_page_index:
                self.thumbnail_canvas.create_rectangle(
                    x - 3, self.thumbnail_margin - 3,
                    x + self.thumbnail_width + 3, self.thumbnail_margin + self.thumbnail_height + 3,
                    outline="#0066FF", width=2
                )
            
            # Draw thumbnail
            if page.get("thumbnail"):
                photo = ImageTk.PhotoImage(page["thumbnail"])
                page["_photo"] = photo  # Store reference to prevent garbage collection
                
                self.thumbnail_canvas.create_image(
                    x + self.thumbnail_width / 2,
                    self.thumbnail_margin + self.thumbnail_height / 2,
                    image=photo
                )
            else:
                # Fallback: draw empty rectangle
                self.thumbnail_canvas.create_rectangle(
                    x, self.thumbnail_margin,
                    x + self.thumbnail_width, self.thumbnail_margin + self.thumbnail_height,
                    outline="black", fill="white"
                )
            
            # Draw page name
            self.thumbnail_canvas.create_text(
                x + self.thumbnail_width / 2,
                self.thumbnail_margin + self.thumbnail_height + 12,
                text=page["name"],
                fill="black",
                font=("Arial", 8)
            )
            
            # Move to next position
            x += self.thumbnail_width + self.thumbnail_margin
            
            # Check if we need to wrap to next row (not implemented in this simplified version)
    
    def _on_thumbnail_click(self, event):
        """Handle click on thumbnail to switch pages"""
        # Calculate which thumbnail was clicked
        x = event.x
        thumbnail_index = (x - self.thumbnail_margin) // (self.thumbnail_width + self.thumbnail_margin)
        
        # Validate index
        if 0 <= thumbnail_index < len(self.pages):
            self.switch_to_page(thumbnail_index)

# Property Panel
Implement dynamic switching mechanisms for different property panels (character, text, image, group).